<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/tmp/spark-and-gis-visualization-on-kaggle-notebooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark with GIS visualization In Kaggle Notebooks.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
required_packages = {"keplergl","geopandas","h3","geocoder"}

import pkg_resources
for lib in required_packages - {pkg.key for pkg in pkg_resources.working_set}:
  print(f"installing {lib}")
  %pip install -q --upgrade pip
  %pip install --use-feature=2020-resolver -q $lib

installing keplergl
     |████████████████████████████████| 1.6 MB 9.2 MB/s 
     |████████████████████████████████| 8.1 MB 6.0 MB/s 
     |████████████████████████████████| 994 kB 43.6 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
     |████████████████████████████████| 6.6 MB 39.3 MB/s 
installing geocoder
     |████████████████████████████████| 98 kB 3.9 MB/s 
installing h3
     |████████████████████████████████| 793 kB 8.6 MB/s 
installing geopandas


In [3]:
%%sh
df

Filesystem     1K-blocks     Used Available Use% Mounted on
overlay        112947452 38870456  74060612  35% /
tmpfs              65536        0     65536   0% /dev
tmpfs            6651464        0   6651464   0% /sys/fs/cgroup
shm              6160384        0   6160384   0% /dev/shm
tmpfs            6651464       36   6651428   1% /var/colab
/dev/sda1       79368748 42981832  36370532  55% /etc/hosts
tmpfs            6651464        0   6651464   0% /proc/acpi
tmpfs            6651464        0   6651464   0% /proc/scsi
tmpfs            6651464        0   6651464   0% /sys/firmware


In [4]:
!sh -c ls

sample_data


In [5]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 65 kB/s 
     |████████████████████████████████| 198 kB 21.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=6cf50b006499be860a3b0200e357fbc6b1c4752f763c8f654b2d95c5fef12960
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [6]:
import pyspark

MAX_MEMORY = "8g"  # 24 gives OOM here.

spark = (pyspark.sql.SparkSession.builder.appName("MyApp") 
    .config("spark.jars.packages", "io.delta:delta-core_2.12:0.8.0") 
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
    .config("spark.executor.memory", MAX_MEMORY) 
    .config("spark.driver.memory", MAX_MEMORY) 
    .enableHiveSupport() 
    .getOrCreate()        
    )
spark

In [7]:
df = spark.createDataFrame([{'hi':'hello','w':'world'}])

In [8]:
 df.write.format('delta').mode('overwrite').option("mergeSchema", "true").save('/tmp/delta1')

In [9]:
df = spark.read.format('delta').load('/tmp/delta1')
df.createOrReplaceTempView('d1')
spark.sql('select * from d1').toPandas()

,hi,w
0,hello,world


In [12]:
import keplergl

In [13]:
map_1 = keplergl.KeplerGl(height=600)
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(height=600)

In [14]:
resize_for_databricks = '''
  <script>
    var targetHeight = "{self.height or 600}px";
    window.setInterval(function() {{
      if (document.body && document.body.style && document.body.style.height !== targetHeight) {{
        document.body.style.height = targetHeight;
      }}
    }}, 2500)
    </script>
    '''
orig_html   = keplergl.KeplerGl._repr_html_(map_1)
better_html = (str(orig_html,'utf-8') + resize_for_databricks)

import base64
b64 = base64.b64encode(better_html.encode('utf-8'))
src = f"data:text/html;base64,{b64.decode('utf-8')}"
base64d_html = f'<iframe src="{src}" style="width:100%; height: 600px">'


In [15]:

from IPython.core.display import HTML
HTML(base64d_html)